In [1]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm, trange 
import os
import glob
import traceback
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


# to display maximum rows and columns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 100)

In [ ]:
chunks_path = r"C:\Sourav\BANKNIFTY\Monthly\Greeks\\"
for file in tqdm(next(os.walk(chunks_path))[2]):
    df = pd.read_csv(chunks_path + file)
    df.to_csv(r"C:\Sourav\BANKNIFTY\Monthly\GreeksCombined\\BANKNIFTY-I.csv", mode='a',
              header=not os.path.exists(r"C:\Sourav\BANKNIFTY\Monthly\GreeksCombined\\BANKNIFTY-I.csv",),
              index=False)

In [ ]:
df = pd.read_csv(r"C:\Sourav\BANKNIFTY\Monthly\GreeksCombined\BANKNIFTY-I.csv")
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df['Time'] = pd.to_datetime(df['Time']).dt.time
df = df.sort_values(by=['Date', 'Time', 'Option_Type', 'Final_strike']).reset_index(drop=True)
df.to_csv(r"C:\Sourav\BANKNIFTY\Monthly\GreeksCombined\BANKNIFTYFinal-I.csv", index=False)

In [7]:
# file paths
output_path = r"D:\Greeks\WeeklyI\5-min\AtTheMoney\\"
#underlying_path = r"E:\sourav\strikeSelector\strikeSelector\input\5Years\EQTill04Jul2022\AllSymbols\\"
dispersion_path = r"D:\Greeks\WeeklyI\5-min\Greeks\\"
#equity_path = r"E:\sourav\strikeSelector\strikeSelector\input\5Years\EQTill04Jul2022\AllSymbols\\"
#misc_path = "D:\Sourav\1-Min\Misc\\"

# select delta values
# delta_list = [0.05, 0.10, 0.15, 0.20, 0.25,
#               0.30, 0.35, 0.40, 0.45, 0.50, 
#               0.55, 0.60, 0.65, 0.70, 0.75,
#               0.80, 0.85, 0.90, 0.95]
delta_list = [0.50]

all_files = sorted(next(os.walk(dispersion_path))[2])
#all_underlying_files = os.listdir(underlying_path)
print(len(all_files))
#print(len(all_underlying_files))

8


In [8]:
# files to be ignored
#ignore_files = ['AUBANK-I.csv', 'AUBANK-II.csv', 'AUBANK-III.csv']

#all_files = sorted(list(set(all_files) - set(ignore_files)))

#print(len(all_underlying_files))
all_files = all_files
all_files


['BankNifty_WeeklyI-2016.csv',
 'BankNifty_WeeklyI-2017.csv',
 'BankNifty_WeeklyI-2018.csv',
 'BankNifty_WeeklyI-2019.csv',
 'BankNifty_WeeklyI-2020.csv',
 'BankNifty_WeeklyI-2021.csv',
 'BankNifty_WeeklyI-2022.csv',
 'BankNifty_WeeklyI-2023.csv']

In [9]:
# select particular files
#all_files = ['IDFCFIRSTB-I.csv', 'IDFCFIRSTB-II.csv', 'IDFCFIRSTB-III.csv']

print(len(all_files))
#all_underlying_files = ['ACC.EQ-NSE.csv']

8


In [13]:
filesProcessed = 0
filesNotProcessed = 0
notProcessedFiles = []

# load main csv
for filename in tqdm(all_files):
    print('Start time : ', datetime.now())
    df = pd.read_csv(dispersion_path + filename)

    df = df.rename(columns={'tickername' : 'Ticker'})
#     df.drop(columns=df.columns[0], axis=1, inplace=True)
    
    # change date column to datetime format
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
    
    # remove 'Unnamed' columns in a csv
    remove_cols = [col for col in df.columns if 'Unnamed' in col]
    df.drop(remove_cols, axis='columns', inplace=True)   

    #print(df.iloc[0:500])
    print('filename :', filename)
    
    # change date column to datetime format
    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

    # replace $ character in a ticker with empty string ('')
    df['Ticker'].replace({'$': None},inplace =True)
    
    
    df['Old_Delta'] = df['Delta'].copy()
    print(df['Delta'].dtype)
    if df['Delta'].dtype == object:
        print('Delta column is in string format.')
        df['Delta'] = df['Delta'].str.replace('\(|\)', '')
        df['Delta'] = df['Delta'].str.replace('\+0j', '')
        df['Delta'] = df['Delta'].str.replace('0j', '0')
        #df.to_csv(output_path + 'IDFCFIRSTB.csv', index=False)
        df['Delta'] = df['Delta'].astype(float)
    
    
    #df = df.drop(['Strike'], axis=1)
    
    df.rename(columns={
                       'Adj_Open' : 'OPT_Open',
                       'Adj_High' : 'OPT_High',
                       'Adj_Low' : 'OPT_Low',
                       'Adj_Close' : 'OPT_Close',
                       'Adj_Volume' : 'OPT_Contracts',
                       'New_OI' : 'OPT_OI',
                       'Final_strike' : 'Strike'
                                            }, inplace=True)
    df2 = df.copy()
    

    # extract Strike and Option Type from the Ticker if it's not there in the input file already.
    
#     df['Strike'] = df['Ticker'].str.extract('([0-9]+[./]*[0-9]*)').astype(float)
    
#     df['Option_Type'] = df["Ticker"].str.split('-').str[0].str[-2:]

#     df['Option_Type'] = np.where((df['Option_Type'] == 'CE') | (df['Option_Type'] == 'PE'),
#                                  df['Option_Type'], 'XX')

    
    # get the symbol from filename
    symbolFilename = filename.split('.')[0]
    print('symbolFilename :', symbolFilename)
    
    # remove '-I'/'-II'/'-III' from filename
    symbol = symbolFilename.replace('-III', '')
    symbol = symbol.replace('-II', '')
    symbol = symbol.replace('-I', '')
    print('symbol :', symbol)
    
    # substitute any special characters ('-', '&', '_') in Dispersion file names with empty string ('_')
    charactersDroppedSymbol = re.sub('\ |\_|\.|\-|\&|\;|\:', '', symbol)

    df2 = df.copy()
    
    ''' 
    for eqFilename in all_underlying_files:
    
        # replace any special characters ('-', '&', '_') in EQ file names with empty string ('')
        charactersDroppedEQFilename = re.sub('\_|\-|\&', '', eqFilename.split('.')[0]).replace('$', '')
        
#         print('charactersDroppedEQFilename :', charactersDroppedEQFilename)
#         print('charactersDroppedSymbol :', charactersDroppedSymbol)
        
        # check if symbol and EQ filenames are same after special characters being dropped 
        if (charactersDroppedEQFilename == charactersDroppedSymbol):
            
            print('eqFilename :', eqFilename)
            
            df_underlying = pd.read_csv(equity_path + eqFilename)
            
            df_underlying.rename(columns={'Open' : 'EQ_Open',
                                          'High' : 'EQ_High',
                                          'Low' : 'EQ_Low',
                                          'Close' : 'EQ_Close',
                                          'Volume' : 'EQ_Volume'}, inplace=True)
            df_underlying['Date'] = pd.to_datetime(df_underlying['Date'], dayfirst=True)
    
    # merge main csv and underlying csv
    try:
        df2 = df.merge(df_underlying[['Date', 'EQ_Open', 'EQ_High', 'EQ_Low', 'EQ_Close']], on=['Date'], how='left')
        df_dates = df['Date'].unique()
        df_underlying_dates = df_underlying['Date'].unique()
        diff_dates = np.setdiff1d(df_dates, df_underlying_dates)
        print('Dates missing in EQ file : ', diff_dates)
        df_underlying = pd.DataFrame()
        
    except Exception as e1:
        
        #print('e :', e1)
        print('Symbol not found in EQ folder : ', filename)
        notProcessedFiles.append(filename)
        filesNotProcessed += 1
        continue    
    '''

    # get the hyphen type (-I, -II, -III) for grouping based on current
    #df2['Ticker_Type'] = df2['Ticker'].str.split('-').str[1]
    
    # calculate difference between 'EQ_Close' and 'Strike' and get the minimum difference for a group
    df2['Time'] = pd.to_datetime(df2['Time']).dt.time
    
    
    exp_df = pd.read_csv(r"E:\sourav\Database\Codes\CsvFiles\WeeklyExpiry.csv")
    exp_df = exp_df.rename(columns={'date' : 'Date',
                                    'Weekly_Expiry_Date' : 'Expiry_Date'})
    exp_df['Date'] = pd.to_datetime(exp_df['Date'], dayfirst=True)
    
    df2 = df2.merge(exp_df[['Date', 'Expiry_Date']], on=['Date'], how='left') 

    df2['Difference'] = abs(df2['EQ_Close'] - df2['Strike'].astype(float))
    dfg = df2.groupby(['Date', 'Time', 'Option_Type'])['Difference']
    df2['Min'] = dfg.transform('min')
    
    # delete output file if it already exists
    try:
        os.remove(output_path + symbolFilename + '.csv')
    except Exception as e:
        #print('e1 : ', e)
        pass
    
    # get 'At The Money' for each group
    dfg = df2.groupby(['Date', 'Time', 'Option_Type'])
    for name, group in tqdm(dfg):
        
        # get 'At The Money' for each group
        try:
            atTheMoney = max(group[(group['Difference'] == group['Min'])]['Strike'])
            group['At_The_Money'] = atTheMoney
        
        except Exception as e:
            #print('e3 : ', e)
            group['At_The_Money'] = np.nan
        
        if symbol != 'BANKNIFTY':
            group = group[['Date', 'Time', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'First_OI', 'Last_OI',
                       'Strike', 'Option_Type', 'Expiry_Date', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega', 'EQ_Open', 
                       'EQ_High', 'EQ_Low', 'EQ_Close', 'At_The_Money']]
            
        else:
            group = group[['Date', 'Time', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI',
                       'Strike', 'Option_Type', 'Expiry_Date', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega', 'EQ_Open', 
                       'EQ_High', 'EQ_Low', 'EQ_Close', 'At_The_Money']]
        
        
        # loop through different delta values
        for delta in delta_list:
            
            # convert delta into string
            # delta = str(int(delta * 100))
            
            if delta == 0.50:
                group.rename(columns={'Min' : f'Delta_{delta*100:.0f}_Diff_Min',
                                      'Difference' : f'Delta_{delta*100:.0f}_Diff'}, 
                             inplace=True)
                group[f'Delta_{delta*100:.0f}_Strike'] = group['At_The_Money']
            else:
                try:
                    if group['Option_Type'].iloc[0] == 'CE':
                        group[f'Delta_{delta*100:.0f}_Diff'] = abs(group['Delta'] - delta)
                    elif group['Option_Type'].iloc[0] == 'PE':
                        group[f'Delta_{delta*100:.0f}_Diff'] = abs(group['Delta'] - (-delta))
                    elif group['Option_Type'].iloc[0] == 'XX':
                        group[f'Delta_{delta*100:.0f}_Diff'] = np.nan

                    group[f'Delta_{delta*100:.0f}_Diff_Min'] = group[f'Delta_{delta*100:.0f}_Diff'].min()        

                    try:
                        deltaStrike = max(group[group[f'Delta_{delta*100:.0f}_Diff'] == group[f'Delta_{delta*100:.0f}_Diff_Min']]['Strike'])
                        group[f'Delta_{delta*100:.0f}_Strike'] = deltaStrike
                    except Exception as e:
                        #print('e2 : ', e)
                        group[f'Delta_{delta*100:.0f}_Strike'] = np.nan

                    # dropping unnecessary columns
                    group = group.drop([f'Delta_{delta*100:.0f}_Diff_Min', f'Delta_{delta*100:.0f}_Diff'], axis=1)
                    
                except Exception as e1:
                    print('e1 : ', e1)
                
#         select required columns in the output csv   
#         if symbol != 'BANKNIFTY':
#             group = group[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI',
#                        'Strike', 'Option_Type', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega', 'EQ_Open', 
#                        'EQ_High', 'EQ_Low', 'EQ_Close', 'EQ_Volume', 'At The Money', f'Delta_{delta*100:.0f}_Diff', f'Delta_{delta*100:.0f}_Diff_Min',
#                        f'Delta_{delta*100:.0f}_Strike', 'Ticker_Type']]
#         else:
#             group = group[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI',
#                        'Strike', 'Option_Type', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega', 'EQ_Open', 
#                        'EQ_High', 'EQ_Low', 'EQ_Close', 'At The Money', f'Delta_{delta*100:.0f}_Diff', f'Delta_{delta*100:.0f}_Diff_Min',
#                        f'Delta_{delta*100:.0f}_Strike', 'Ticker_Type']]

        # write output to csv
        group.to_csv(output_path + symbolFilename + '.csv', mode='a', header=not os.path.exists(output_path + symbolFilename + '.csv'), index=False)
    filesProcessed += 1
    print('End time : ', datetime.now())

print('Files processed :', filesProcessed)
print('Files not processed :', filesNotProcessed)  
print('Not processed files :', notProcessedFiles)



  0%|          | 0/8 [00:00<?, ?it/s]

Start time :  2023-05-03 12:34:49.795608
filename : BankNifty_WeeklyI-2016.csv
float64
symbolFilename : BankNifty_WeeklyI-2016
symbol : BankNifty_WeeklyI-2016


  0%|          | 0/22200 [00:00<?, ?it/s]

End time :  2023-05-03 12:48:31.267758
Start time :  2023-05-03 12:48:31.267758
filename : BankNifty_WeeklyI-2017.csv
float64
symbolFilename : BankNifty_WeeklyI-2017
symbol : BankNifty_WeeklyI-2017


  0%|          | 0/36990 [00:00<?, ?it/s]

...

End time :  2023-05-03 14:44:15.087357
Start time :  2023-05-03 14:44:15.087357
filename : BankNifty_WeeklyI-2022.csv
float64
symbolFilename : BankNifty_WeeklyI-2022
symbol : BankNifty_WeeklyI-2022


  0%|          | 0/37050 [00:00<?, ?it/s]

End time :  2023-05-03 15:07:49.300799
Start time :  2023-05-03 15:07:49.300799
filename : BankNifty_WeeklyI-2023.csv
float64
symbolFilename : BankNifty_WeeklyI-2023
symbol : BankNifty_WeeklyI-2023


  0%|          | 0/9600 [00:00<?, ?it/s]

End time :  2023-05-03 15:13:10.899407
Files processed : 8
Files not processed : 0
Not processed files : []


In [ ]:
from datetime import datetime
x1 = pd.read_csv(r"E:\sourav\Database\1-Min\Greeks\SBIN-I.csv")
print(x1.shape[0])
x1['Date'] = pd.to_datetime(x1['Date'], dayfirst=True)
date1 = datetime.strptime('30-01-2020', '%d-%m-%Y')
date2 = datetime.strptime('31-01-2020', '%d-%m-%Y')

x2 = x1[(x1['Date']==date2)]
display(x2)

In [ ]:
x2['Expiry_Date'].unique()

In [ ]:
x1 = pd.read_csv(r"D:\Sourav\1-Min\Greeks\HDFCBANK-I.csv")
x1['imag'] = np.imag(x1.Delta)
x1['real'] = np.real(x1.Delta)
display(x1)

In [ ]:
x1['Delta'].str.replace('\(|\)|\+0j', '')

In [ ]:
x2 = pd.DataFrame({'Num' : [10+13j, 15+0j, 15]})
x2

## Append -I, -II and -III files 

In [ ]:
input_path = r"D:\Sourav\EOD\05-50DeltaStrikesAddedWithNewClose\\"
output_path = r"D:\Sourav\BankNiftyConstituents2011-2015-Merged\\"
symbols = next(os.walk(input_path))[2]

for i in tqdm(range(len(symbols))):
    symbols[i] = symbols[i].replace('-III.csv', '').replace('-II.csv', '').replace('-I.csv', '')
    
symbols = sorted(set(symbols))
symbols = ['ICICIBANK']
symbols

In [ ]:
hyphens = ['-I', '-II', '-III']

for symbol in tqdm(symbols):
    df = pd.DataFrame()
    sum1 = 0
    for i in hyphens:
        file_path = input_path + symbol + i + '.csv'
        if os.path.isfile(file_path):
            df1 = pd.read_csv(file_path)
            df = df.append(df1)
        df.to_csv(output_path + symbol + '.csv', index=False)
        sum1 += df1.shape[0]
    print(sum1)
    print(symbol + ' : ' + str(df.shape[0]))

In [ ]:
hyphens = ['-I', '-II', '-III']

for symbol in tqdm(symbols):
    df = pd.DataFrame()
    sum1 = 0
    for i in hyphens:
        file_path = input_path + symbol + i + '.csv'
        if os.path.isfile(file_path):

In [ ]:
input_path = r"D:\Sourav\BN\05-50DeltaStrikesAdded\\"
new_close_path = r"D:\purvang\BN\New Close\\"
output_path = r"D:\Sourav\BN\05-50DeltaStrikesAddedFutMonthly\\"

symbols = next(os.walk(input_path))[2]
#symbols = ['AUBANK-I.csv', 'AUBANK-II.csv', 'AUBANK-III.csv']

for i in tqdm(range(len(symbols))):
    symbols[i] = symbols[i].replace('-III.csv', '').replace('-II.csv', '').replace('-I.csv', '')
    
symbols = sorted(set(symbols))
symbols

In [ ]:
hyphens = ['-I', '-II', '-III']

for symbol in tqdm(symbols):
    df = pd.DataFrame()
    sum1 = 0
    print(symbol)
    for i in tqdm(hyphens):
        print(i)
        file_path = input_path + symbol + i + '.csv'
        df = pd.read_csv(file_path)
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        df['Ticker'] = np.where(df['Option_Type']=='XX', symbol + 'Fut_Monthly' + ' - ' + i[1:],
                                df['Ticker'])
        file_path = new_close_path + '_' + symbol + i + '_adj.csv.csv'
        if os.path.isfile(file_path):
            new_close_df = pd.read_csv(file_path)
            
            new_close_df['Date'] = pd.to_datetime(new_close_df['Date'], dayfirst=True)
            
            new_close_df['Remainder'] = new_close_df['Adj_strike'] % new_close_df['Adj_strike'].astype(int)
            
            new_close_df['Ticker'] = np.where(new_close_df['Remainder']==0, 
                            symbol + new_close_df['Adj_strike'].astype(int).astype(str) + new_close_df['OPTION_TYP'] + ' - ' + i[1:],
                            symbol + new_close_df['Adj_strike'].astype(float).astype(str) + new_close_df['OPTION_TYP'] + ' - ' + i[1:])            
            
            new_close_df['Ticker'] = np.where(new_close_df['OPTION_TYP']=='XX', symbol + 'Fut_Monthly' + ' - ' + i[1:], 
                                              new_close_df['Ticker'])
            
            df = df.drop(['OPT_Close'], axis=1)
            new_close_df = new_close_df.drop(['Remainder'], axis=1)
            df = df.merge(new_close_df[['Date', 'Ticker', 'new_close']], on=['Date', 'Ticker'], how='left')
            
            
            df = df.rename(columns={'new_close' : 'OPT_Close'})
            df = df[['Date', 'Ticker', 'OPT_Open', 'OPT_High', 'OPT_Low', 'OPT_Close', 'OPT_Contracts', 'OPT_OI', 
                     'Strike', 'Option_Type', 'EXPIRY_DT', 'IV', 'Delta', 'Theta', 'Gamma', 'Vega',  
                     'EQ_Open',  'EQ_High',   'EQ_Low',  'EQ_Close',  'At_The_Money',  'Delta_5_Strike',  
                     'Delta_10_Strike', 'Delta_15_Strike',  'Delta_20_Strike',  'Delta_25_Strike',  
                     'Delta_30_Strike', 'Delta_35_Strike', 'Delta_40_Strike',  'Delta_45_Strike', 'Delta_50_Strike']]
            df.to_csv(output_path + symbol + i + '.csv' ,index=False)
        